In [148]:
import torch
import string
import json

In [2]:
from transformers import BertTokenizer, BertForMaskedLM
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()

top_k = 10

C:\Users\shahy\anaconda3\lib\site-packages\transformers\configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClas

In [3]:
def decode(tokenizer, pred_idx, top_clean):
    ignore_tokens = string.punctuation + '[PAD]'
    tokens = []
    for w in pred_idx:
        token = ''.join(tokenizer.decode(w).split())
        if token not in ignore_tokens:
            tokens.append(token.replace('##', ''))
    return '\n'.join(tokens[:top_clean])

In [4]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
    text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
    # if <mask> is the last token, append a "." so that models dont predict punctuation.
    if tokenizer.mask_token == text_sentence.split()[-1]:
        text_sentence += ' .'

    input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
    mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
    return input_ids, mask_idx

In [178]:
def get_predictied_word(res, top_clean=10):
    # ========================= BERT =================================
    i=0
    c = 0
    list_words=[]
    a = res[0]
    output_text=''
    if a!="":
        for w in res:
            if w!='<mask>':
                input_text = a +' <mask>'
                input_ids, mask_idx = encode(bert_tokenizer, input_text)
                with torch.no_grad():
                    predict = bert_model(input_ids)[0]
                bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)
                if (i+1) < len(res):
                    output_text = res[i+1]
                    if output_text in bert:
                        list_words.append(output_text)
                        #print(list_words)
                        c+=1
                    
            a = a+' '+output_text 
            #print(a)
            i = i+1
        
    len_predicted_words = len(list_words)   
    return c,'\n'.join(list_words[:len_predicted_words])

In [186]:
def get_all_predictions(text_sentence, top_clean=10):
    # ========================= BERT =================================
    input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
    word_predicted=''
    accuracy=0
    with torch.no_grad():
        predict = bert_model(input_ids)[0]
    res = len(text_sentence.split())
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)
    c,word_predicted = get_predictied_word(text_sentence.split())
    if (res-2)>0: 
        accuracy = (c/(res-2)) * 100
    
    return {'bert': bert,'input_length':res-1, 'predicted_words_used':c,'predictied_words_used':word_predicted,'accuracy':accuracy}


In [190]:
abc = get_all_predictions("machine learning research paper <mask>")
print(abc)

{'bert': 'no\nseries\nreview\nvol\nsociety\njournal\nselection\nnr\nbulletin\nproceedings', 'input_length': 4, 'predicted_words_used': 2, 'predictied_words_used': 'learning\nresearch', 'accuracy': 66.66666666666666}
